## Import neccessary libraries

In [ ]:
# Enable automatic reloading of modules
%reload_ext autoreload
%autoreload 2

# Standard Library Imports
import os
from natsort import natsorted

# Third-Party Library Imports
import numpy as np
import pickle

# DFTTK Imports
from dfttk.eos.functions import BM4_derivative, BM4_derivative2
from dfttk.configuration import Configuration as DFTTKConfiguration

# PyZentropy imports
from pyzentropy.configuration import Configuration as PyZentropyConfiguration
from pyzentropy.system import System

# DFTTK Configuration

## Energy-volume curves

In [ ]:
# Define the path to the configs folder and generate the config names
configs_path = "Fe3Pt/configs" 
config_names = [f for f in os.listdir(configs_path) if os.path.isdir(os.path.join(configs_path, f)) and f.startswith("config_")]
config_names = natsorted(config_names)

In [ ]:
# Create the DFTTK configuration objects
vasp_cmd = ["mpirun", "/opt/packages/VASP/VASP6/6.4.3/ONEAPI/vasp_std"]
multiplicity = np.array([2, 12, 6])
config_objects = {name: DFTTKConfiguration(os.path.join(configs_path, name), name, vasp_cmd, "FCC Fe3Pt 12-atom supercell", multiplicity[i]) for i, name in enumerate(config_names)}

In [ ]:
# Process the EV curves for each configuration
for config_name in config_names:
    config_objects[config_name].process_ev_curve(collect_mag_data=True)

## Debye-Grüneisen model

In [ ]:
volumes = np.linspace(0.98*139, 1.02*172, 1000)
temperatures = np.arange(0, 1010, 10)

# Process the Debye model for each configuration
for name in config_names:
    config_objects[name].process_debye(volumes=volumes, temperatures=temperatures, scaling_factor=0.617, gruneisen_x=2/3)

## Quasiharmonic approximation

In [ ]:
# Process the QHA for each configuration
for config_name in config_names:
    config = config_objects[config_name]
    config.process_qha("debye", P = 0)

# PyZentropy Configuration

Use outputs from DFTTK Configuration objects as inputs to PyZentropy Configuration objects.

In [ ]:
for name in config_names:
    # Prepare lists to store first and second derivatives for each temperature
    helmholtz_energies_derivatives_list = []
    helmholtz_energies_derivatives2_list = []

    for temperature in temperatures:
        # Access nested dictionary levels for EOS constants
        methods = config_objects[name].qha.methods
        debye = methods['debye']
        helmholtz_energy = debye['helmholtz_energy']
        eos_constants = helmholtz_energy['eos_constants'][f'{temperature}K']

        # Extract EOS parameters
        b = eos_constants['b']
        c = eos_constants['c']
        d = eos_constants['d']

        # Compute first and second derivatives using BM4 functions
        derivatives = BM4_derivative(volumes, b, c, d)
        derivatives2 = BM4_derivative2(volumes, b, c, d)

        # Store results for this temperature
        helmholtz_energies_derivatives_list.append(derivatives)
        helmholtz_energies_derivatives2_list.append(derivatives2)

    # Convert lists to 2D NumPy arrays and store in config object
    helmholtz_energies_derivatives = np.array(helmholtz_energies_derivatives_list)
    helmholtz_energies_derivatives2 = np.array(helmholtz_energies_derivatives2_list)
    config_objects[name].qha.methods['debye']['helmholtz_energy']['derivatives'] = helmholtz_energies_derivatives
    config_objects[name].qha.methods['debye']['helmholtz_energy']['derivatives2'] = helmholtz_energies_derivatives2
    

In [ ]:
# Initialize dictionary to hold PyZentropy Configuration objects
new_config_objects = {}
reference_helmholtz_energies = config_objects["config_0"].qha.methods['debye']['helmholtz_energy']['values'][1:, :]

for i, name in enumerate(config_names):
    config = config_objects[name]

    # Remove 0 K results from each configuration.
    # TODO: we want to include this in the near future
    # Create a Configuration object for each config and store in the dictionary
    new_config_objects[name] = PyZentropyConfiguration(
        name=name,
        multiplicity=config.multiplicity,
        number_of_atoms=config.qha.number_of_atoms,
        volumes=config.qha.volumes,
        temperatures=config.qha.temperatures[1:],
        # Extract Helmholtz energies and their derivatives from DFTTK results
        helmholtz_energies=config.qha.methods['debye']['helmholtz_energy']['values'][1:, :],
        helmholtz_energies_dV=config.qha.methods['debye']['helmholtz_energy']['derivatives'][1:, :],
        helmholtz_energies_d2V2=config.qha.methods['debye']['helmholtz_energy']['derivatives2'][1:, :],
        reference_helmholtz_energies=reference_helmholtz_energies,
        # Extract entropy and heat capacity data
        entropies=config.qha.methods['debye']['entropy']['values'][1:, :],
        heat_capacities=config.qha.methods['debye']['heat_capacity']['values'][1:, :]
    )

In [ ]:
# Rename config names for clarity
new_config_objects = {
    "FM": new_config_objects["config_0"],
    "SF22": new_config_objects["config_22"],
    "SF28": new_config_objects["config_28"]
}

# ALso set the config.name attribute correctly
for name, config in new_config_objects.items():
    config.name = name

In [ ]:
# Save the PyZentropy Configuration objects to a pickle file to use for testing
with open("Fe3Pt_three_configs.pkl", "wb") as f:
    pickle.dump(new_config_objects, f)

In [ ]:
# Plotting - smoke test
new_config_objects["FM"].plot_vt("helmholtz_energy_vs_volume")

In [ ]:
# Plotting - selected temperatures and volumes 
selected_temperatures = np.array([300, 400, 500, 600, 700, 800, 900, 1000])
selected_volumes = np.array([100, 150, 200])
new_config_objects["FM"].plot_vt("helmholtz_energy_vs_temperature", selected_temperatures, selected_volumes)

In [ ]:
# Plotting - invalid plot type should raise an error
#new_config_objects["FM"].plot_vt("invalid_plot_type")

In [ ]:
# Plotting - set internal_energies, entropies, and heat_capacities to None - should raise an error
#new_config_objects["FM"].internal_energies = None
#new_config_objects["FM"].entropies = None
#new_config_objects["FM"].heat_capacities = None
#new_config_objects["FM"].plot_vt("heat_capacity_vs_temperature")

# PyZentropy System

Create a PyZentropy System object using the Configuration objects and inputs and calculate the thermodynamic properties.

In [ ]:
# Create a System object from the configuration objects
# Thermodynamic calculations for the system will be performed automatically during initialization
system = System(new_config_objects, reference_helmholtz_energies)

In [ ]:
# Calculate phase diagrams
system.calculate_phase_diagrams(ground_state="FM", dP=0.5)

In [ ]:
# Save system object for testing
with open("Fe3Pt_system.pkl", "wb") as f:
    pickle.dump(system, f)

In [ ]:
# Options - helmholtz_energy_vs_volume, helmholtz_energy_vs_temperature, 
# helmholtz_energy_dV_vs_volume, helmholtz_energy_dV_vs_temperature,
# helmholtz_energy_d2V2_vs_volume, helmholtz_energy_d2V2_vs_temperature
# entropy_vs_volume, entropy_vs_temperature
# configurational_entropy_vs_volume, configurational_entropy_vs_temperature
# heat_capacity_vs_volume, heat_capacity_vs_temperature
# bulk_modulus_vs_volume, bulk_modulus_vs_temperature
# vt_phase_diagram
selected_temperatures = np.array([300, 400, 500, 600, 700, 800, 900, 1000])
selected_volumes = np.array([100, 150, 200])
system.plot_vt("entropy_vs_volume")
#system.plot_vt("helmholtz_energy_vs_temperature", selected_temperatures=selected_temperatures, selected_volumes=selected_volumes)

In [ ]:
# Options - helmholtz_energy_pt_vs_volume
selected_temperatures = np.array([300, 400, 500, 600, 700, 800, 900, 1000])
system.plot_pt("volume_vs_temperature", P=0, selected_temperatures=selected_temperatures, ground_state="FM")

# volume_vs_temperature, CTE_vs_temperature, LCTE_vs_temperature
# entropy_vs_temperature, configurational_entropy_vs_temperature
# heat_capacity_vs_temperature, bulk_modulus_vs_temperature
# gibbs_energy_vs_temperature, probability_vs_temperature
# pt_phase_diagram